In [27]:
import pandas as pd
import numpy as np
import h3
import random
from geopy.distance import geodesic
from scipy.stats import multivariate_normal

def h3_simulation(N=20, resolution=9, temporal_interval_minutes=30, vehicle_speed_kmh=20, min_distance_h3_units=3):
    """
    Generate N ride requests in San Francisco City Region for DARP simulation.
    
    This function creates realistic ride-sharing requests by:
    1. Defining San Francisco's geographic boundaries using H3 hexagonal indexing
    2. Generating pickup and dropoff locations from a 2D Gaussian distribution
    3. Ensuring spatial constraints (minimum distance between pickup/dropoff)
    4. Calculating realistic temporal constraints based on travel time and vehicle speed
    5. Assigning time windows within a 24-hour period divided into intervals
    
    Parameters:
    -----------
    N : int, default=20
        Number of ride requests to generate
    resolution : int, default=9
        H3 resolution level (higher = smaller hexagons, more precise)
    temporal_interval_minutes : int, default=30
        Time interval duration in minutes (24h is divided by this)
    vehicle_speed_kmh : float, default=30
        Average vehicle speed in km/h for travel time calculations
    min_distance_h3_units : int, default=3
        Minimum distance between pickup and dropoff in H3 resolution units
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: origin, destination, o_t_index, d_t_index
        - origin: Origin (pickup) H3 index
        - destination: Destination (dropoff) H3 index  
        - o_t_index: Origin time window index (1-48 for 30min intervals)
        - d_t_index: Destination time window index (1-48 for 30min intervals)
    """
    
    # San Francisco inner city boundaries (lat, lon) - more restrictive to stay within city limits
    sf_center_lat, sf_center_lon = 37.7749, -122.4194
    sf_bounds = {
        'north': 37.8100,
        'south': 37.7300, 
        'east': -122.3700,
        'west': -122.5000
    }
    
    # Generate all H3 indices covering San Francisco at given resolution
    sf_h3_indices = []
    
    # Create a grid of points within SF bounds and get their H3 indices
    lat_range = np.linspace(sf_bounds['south'], sf_bounds['north'], 50)
    lon_range = np.linspace(sf_bounds['west'], sf_bounds['east'], 50)
    
    for lat in lat_range:
        for lon in lon_range:
            h3_index = h3.latlng_to_cell(lat, lon, resolution)
            if h3_index not in sf_h3_indices:
                sf_h3_indices.append(h3_index)
    
    # Calculate number of time intervals in 24 hours
    num_time_intervals = int(24 * 60 / temporal_interval_minutes)  # 48 for 30min intervals
    
    # Set up 2D Gaussian distribution centered on SF
    mean = [sf_center_lat, sf_center_lon]
    # Covariance matrix - adjust spread to cover SF inner city area
    cov = [[0.005, 0], [0, 0.005]]  # Smaller spread to stay within inner city bounds
    
    requests = []
    
    for _ in range(N):
        attempts = 0
        max_attempts = 100
        
        while attempts < max_attempts:
            # Sample pickup and dropoff locations from 2D Gaussian
            pickup_lat, pickup_lon = multivariate_normal.rvs(mean, cov)
            dropoff_lat, dropoff_lon = multivariate_normal.rvs(mean, cov)
            
            # Convert to H3 indices
            pickup_h3 = h3.latlng_to_cell(pickup_lat, pickup_lon, resolution)
            dropoff_h3 = h3.latlng_to_cell(dropoff_lat, dropoff_lon, resolution)
            
            # Check if both locations are within SF bounds
            if (pickup_h3 in sf_h3_indices and dropoff_h3 in sf_h3_indices and 
                pickup_h3 != dropoff_h3):
                
                # Check minimum distance constraint
                pickup_center = h3.cell_to_latlng(pickup_h3)
                dropoff_center = h3.cell_to_latlng(dropoff_h3)
                
                # Calculate distance in km
                distance_km = geodesic(pickup_center, dropoff_center).kilometers
                # Calculate H3 unit distance (approximate)
                h3_edge_length_km = h3.average_hexagon_edge_length(resolution, unit='km')
                distance_h3_units = distance_km / h3_edge_length_km
                
                if distance_h3_units >= min_distance_h3_units:
                    # Calculate minimum travel time in intervals
                    travel_time_hours = distance_km / vehicle_speed_kmh * 2

                    travel_time_intervals = int(np.ceil(travel_time_hours * 60 / temporal_interval_minutes)) + 2
                    
                    # Generate pickup time window (1 to num_time_intervals)
                    # Ensure enough time remains for dropoff
                    max_pickup_time = num_time_intervals - travel_time_intervals
                    if max_pickup_time > 0:
                        pickup_time_index = random.randint(1, max_pickup_time)
                        dropoff_time_index = pickup_time_index + travel_time_intervals
                        
                        # Ensure dropoff time is within valid range
                        if dropoff_time_index <= num_time_intervals:
                            requests.append({
                                'origin': pickup_h3,
                                'destination': dropoff_h3,
                                'o_t_index': pickup_time_index,
                                'd_t_index': dropoff_time_index
                            })
                            break
            
            attempts += 1
        
        if attempts >= max_attempts:
            print(f"Warning: Could not generate valid request after {max_attempts} attempts")
    
    return pd.DataFrame(requests)


In [28]:
requests = h3_simulation()
requests

,origin,destination,o_t_index,d_t_index
0,89283082d0bffff,89283095a1bffff,9,12
1,892830829a3ffff,892830875cfffff,34,37
2,89283082167ffff,89283082c93ffff,2,5
3,89283080133ffff,89283082377ffff,14,17
4,8928308708bffff,89283095b23ffff,9,12
5,8928308709bffff,89283082393ffff,17,22
6,89283095367ffff,89283082c4bffff,21,24
7,89283082ed3ffff,89283095c6bffff,20,24
8,89283082a03ffff,89283095a6fffff,15,19
9,89283095a67ffff,89283095e4bffff,16,19


In [19]:
import folium

# Create a folium map centered on San Francisco
sf_center = [37.7749, -122.4194]
m = folium.Map(location=sf_center, zoom_start=12)

# Add origin-destination pairs as arrows
for idx, row in requests.iterrows():
    # Get lat/lon coordinates for origin and destination
    origin_lat, origin_lon = h3.cell_to_latlng(row['origin'])
    dest_lat, dest_lon = h3.cell_to_latlng(row['destination'])
    
    # Add origin marker
    folium.CircleMarker(
        location=[origin_lat, origin_lon],
        radius=5,
        popup=f"Origin {idx}: Time {row['o_t_index']}",
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.7
    ).add_to(m)
    
    # Add destination marker
    folium.CircleMarker(
        location=[dest_lat, dest_lon],
        radius=5,
        popup=f"Destination {idx}: Time {row['d_t_index']}",
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.3
    ).add_to(m)
    
    # Add arrow (line) from origin to destination
    folium.PolyLine(
        locations=[[origin_lat, origin_lon], [dest_lat, dest_lon]],
        color='green',
        weight=2,
        opacity=0.8,
        popup=f"Trip {idx}: {row['o_t_index']} → {row['d_t_index']}"
    ).add_to(m)
    
    # Add arrow head using a small triangle marker at destination
    folium.RegularPolygonMarker(
        location=[dest_lat, dest_lon],
        number_of_sides=3,
        radius=3,
        rotation=0,
        color='green',
        fill=True,
        fillColor='green',
        fillOpacity=0.8
    ).add_to(m)

# Display the map
m
